In [ ]:
import open3d as o3d
import numpy as np


def slice_and_layout_3d_alternative(mesh_path, num_slices=10, axis="z"):
    """
    Slices a 3D mesh into cross-sections and lays them next to each other in 3D.

    Parameters:
    - mesh_path: Path to the STL file.
    - num_slices: Number of slices to generate.
    - axis: Axis for slicing ('x', 'y', 'z').

    Returns:
    - A 3D visualization of the slices laid out next to each other.
    """
    # Step 1: Load the 3D mesh
    mesh = o3d.io.read_triangle_mesh(mesh_path)
    mesh.compute_vertex_normals()

    # Step 2: Determine slicing planes
    bbox = mesh.get_axis_aligned_bounding_box()
    min_bound = bbox.min_bound
    max_bound = bbox.max_bound
    axis_index = {"x": 0, "y": 1, "z": 2}[axis]
    slice_positions = np.linspace(min_bound[axis_index], max_bound[axis_index], num_slices)

    slices = []
    offset = 0

    # Step 3: Slice the mesh manually
    for i, pos in enumerate(slice_positions):
        # Define the lower and upper bounds for this slice
        lower_bound = min_bound.copy()
        upper_bound = max_bound.copy()
        lower_bound[axis_index] = pos
        upper_bound[axis_index] = pos + (max_bound[axis_index] - min_bound[axis_index]) / num_slices

        # Create the slicing box
        slicing_box = o3d.geometry.AxisAlignedBoundingBox(lower_bound, upper_bound)
        sliced_mesh = mesh.crop(slicing_box)

        # Apply an offset to lay the slices next to each other
        if len(sliced_mesh.vertices) > 0:  # Ensure the slice is valid
            offset_vector = np.zeros(3)
            offset_vector[(axis_index + 1) % 3] = offset  # Offset along an orthogonal axis
            sliced_mesh.translate(offset_vector)
            slices.append(sliced_mesh)

            offset += bbox.get_extent()[(axis_index + 1) % 3] / num_slices  # Increment the offset

    # Step 4: Visualize the slices
    o3d.visualization.draw_geometries(slices, window_name="3D Slices Laid Out")


# File path to the provided STL file
mesh_file_path = "stump.stl"  # Replace with your local file path

# Slice the mesh and visualize the slices in 3D
slice_and_layout_3d_alternative(mesh_file_path, num_slices=10, axis="z")